In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install findspark

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=81ae57eb56c4e7dae9795202e218d39bc6aea8d1950b56e46548be421c008698
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [8]:
import findspark
from pyspark.sql.functions import  * # to_date, col, split, char_length, lit, trim
# findspark.init()
from pyspark.sql import SparkSession
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()
# spark = SparkSession.builder.appName("app_name").getOrCreate()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")


df = spark.read.option('multiline', 'true').json("/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/json_files/US_category_id.json")
df.printSchema()
df.show(10)


root
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- assignable: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |-- kind: string (nullable = true)

+--------------------+--------------------+--------------------+
|                etag|               items|                kind|
+--------------------+--------------------+--------------------+
|"m2yskBQFythfE4ir...|[{"m2yskBQFythfE4...|youtube#videoCate...|
+--------------------+--------------------+--------------------+



In [9]:

#Flatten array of structs and structs
def flatten(df):
  #  compute Complex Fields (Lists and Structs) in Schema
   complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   while len(complex_fields)!=0:
      col_name=list(complex_fields.keys())[0]
      print ("Processing :"+col_name+" Type : "+str(type(complex_fields[col_name])))

      # if StructType then convert all sub element to columns.
      # i.e. flatten structs
      if (type(complex_fields[col_name]) == StructType):
         expanded = [col(col_name+'.'+k).alias(col_name+'_'+k) for k in [ n.name for n in  complex_fields[col_name]]]
         df=df.select("*", *expanded).drop(col_name)

      # if ArrayType then add the Array Elements as Rows using the explode function
      # i.e. explode Arrays
      elif (type(complex_fields[col_name]) == ArrayType):
         df=df.withColumn(col_name,explode_outer(col_name))

      # recompute remaining Complex Fields in Schema
      complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if type(field.dataType) == ArrayType or  type(field.dataType) == StructType])
   return df

In [10]:
df_flatten = flatten(df)

Processing :items Type : <class 'pyspark.sql.types.ArrayType'>
Processing :items Type : <class 'pyspark.sql.types.StructType'>
Processing :items_snippet Type : <class 'pyspark.sql.types.StructType'>


In [11]:
df_flatten.show()

+--------------------+--------------------+--------------------+--------+--------------------+------------------------+-----------------------+-------------------+
|                etag|                kind|          items_etag|items_id|          items_kind|items_snippet_assignable|items_snippet_channelId|items_snippet_title|
+--------------------+--------------------+--------------------+--------+--------------------+------------------------+-----------------------+-------------------+
|"m2yskBQFythfE4ir...|youtube#videoCate...|"m2yskBQFythfE4ir...|       1|youtube#videoCate...|                    true|   UCBR8-60-B28hp2Bm...|   Film & Animation|
|"m2yskBQFythfE4ir...|youtube#videoCate...|"m2yskBQFythfE4ir...|       2|youtube#videoCate...|                    true|   UCBR8-60-B28hp2Bm...|   Autos & Vehicles|
|"m2yskBQFythfE4ir...|youtube#videoCate...|"m2yskBQFythfE4ir...|      10|youtube#videoCate...|                    true|   UCBR8-60-B28hp2Bm...|              Music|
|"m2yskBQFythfE4